In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from torch.utils.data import Dataset
import torch

class CustomTextDataset(Dataset):
    def __init__(self, annotations_file, text_column, label_column, tokenizer, max_length):
        self.dataframe = pd.read_csv(annotations_file)
        self.text_column = text_column
        self.label_column = label_column
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx][self.text_column]
        label = self.dataframe.iloc[idx][self.label_column]

        # Tokenization
        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoded_text['input_ids'].flatten(),
            'attention_mask': encoded_text['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [3]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, label_size)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        # Take the output of the last time step
        last_time_step = lstm_out[:, -1, :]
        out_space = self.fc(last_time_step)
        return out_space




In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('imdb')

texts = dataset['train']['text']
labels = dataset['train']['label']

# Creating a DataFrame and saving to CSV
df = pd.DataFrame({'text': texts, 'label': labels})
df.to_csv('imdb_dataset.csv', index=False)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating unsupervised split: 0 examples [00:00, ? examples/s]

In [5]:
df

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [9]:
df['text'][10]

'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about a de

In [10]:
df['label'][10]

0

In [11]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = CustomTextDataset(
    annotations_file='imdb_dataset.csv',  # Replace with your CSV file
    text_column='text',
    label_column='label',
    tokenizer=tokenizer,
    max_length=128
)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
len(dataset)

25000

In [14]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Model initialization
model = LSTMClassifier(
    embedding_dim=128,
    hidden_dim=256,
    vocab_size=len(tokenizer.vocab),
    label_size=2
    )

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# Initialize the optimizer and loss function
optimizer = Adam(model.parameters())
criterion = CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        train_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")


Epoch 1/5, Training Loss: 0.0985
Epoch 2/5, Training Loss: 0.0529
Epoch 3/5, Training Loss: 0.0377


KeyboardInterrupt: ignored